In [1]:
import os
import pandas as pd
from price_loaders.tradingview import load_asset_price
import numpy as np

def save_to_csv(data, folder, filename):
    filepath = os.path.join(folder, filename)
    data.to_csv(filepath, index=False)
    print(f"Lưu dữ liệu thành công vào {filepath}")

In [2]:


# eu_gdp_growth                   = load_asset_price("EUGDPYY", 80, "3M", None) 
# eu_interest_rate                = load_asset_price("EUINTR", 240, "1M", None) 
# eu_inflation_rate               = load_asset_price("EUIRYY", 240, "1M", None)
# eu_consumer_price_index         = load_asset_price("EUCPI", 240, "1M", None)
# eu_producer_price_index         = load_asset_price("EUPPI", 240, "1M", None)
# eu_unemployment_rate            = load_asset_price("EUUR", 240, "1M", None)
# eu_trade_balance                = load_asset_price("EUBOT", 240, "1M", None)
# eu_gov_debt                     = load_asset_price("EUGDG", 10, "12M", None)
# eu_consumer_confidence_index    = load_asset_price("EUCCI", 240, "1M", None)
# eu_retail_sales                 = load_asset_price("EURSMM", 2,"1M", None)
# eu_money_supply                 = load_asset_price("EUM2", 2,"1M", None)



In [28]:
#EURUSD
eurusd_D = load_asset_price("EURUSD", 7560, "1D", None)  # ~30 năm ngày
eurusd_W = load_asset_price("EURUSD", 1560, "1W", None)  # ~30 năm tuần
eurusd_M = load_asset_price("EURUSD", 360, "1M", None)   # ~30 năm tháng

In [9]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

def calculate_swing_high_low(df, timeframe='D'):
    """
    Tính swing high/low và lọc đỉnh/đáy hợp lệ theo logic đã cho.
    Trả về df với 2 cột mới: 'valid_swing_high', 'valid_swing_low' (bool).
    """
    df = df.reset_index(drop=True)

    if(timeframe == 'D'):
        window = 5
    elif(timeframe == 'W'):
        window = 4
    elif(timeframe == 'M'):
        window = 3
    else:
        raise ValueError("Invalid timeframe")
    highs = df['high']
    lows = df['low']

    swing_highs = []
    swing_lows = []
    n = len(df)

    # Tìm điểm swing high/low thô
    for i in range(n):
        start = max(i - window, 0)
        end = min(i + window + 1, n)

        max_around = highs[start:i].max() if i > start else -np.inf
        max_around = max(max_around, highs[i+1:end].max() if i+1 < end else -np.inf)

        min_around = lows[start:i].min() if i > start else np.inf
        min_around = min(min_around, lows[i+1:end].min() if i+1 < end else np.inf)

        swing_highs.append(highs[i] > max_around)
        swing_lows.append(lows[i] < min_around)

    df['swing_high'] = swing_highs
    df['swing_low'] = swing_lows

    swing_high_indices = df.index[df['swing_high']].tolist()
    swing_low_indices = df.index[df['swing_low']].tolist()

    # Hàm kiểm tra swing high hợp lệ
    def is_valid_swing_high(sh_idx):
        lows_before = [idx for idx in swing_low_indices if idx < sh_idx]
        lows_after = [idx for idx in swing_low_indices if idx > sh_idx]
        if not lows_before or not lows_after:
            return False
        low_before_idx = max(lows_before)
        low_after_idx = min(lows_after)
        high_price = df.at[sh_idx, 'high']
        low_before_price = df.at[low_before_idx, 'low']
        low_after_price = df.at[low_after_idx, 'low']
        return low_before_price < high_price and low_after_price < high_price

    # Hàm kiểm tra swing low hợp lệ
    def is_valid_swing_low(sl_idx):
        highs_before = [idx for idx in swing_high_indices if idx < sl_idx]
        highs_after = [idx for idx in swing_high_indices if idx > sl_idx]
        if not highs_before or not highs_after:
            return False
        high_before_idx = max(highs_before)
        high_after_idx = min(highs_after)
        low_price = df.at[sl_idx, 'low']
        high_before_price = df.at[high_before_idx, 'high']
        high_after_price = df.at[high_after_idx, 'high']
        return high_before_price > low_price and high_after_price > low_price

    valid_swing_highs = [(idx, is_valid_swing_high(idx)) for idx in swing_high_indices]
    valid_swing_lows = [(idx, is_valid_swing_low(idx)) for idx in swing_low_indices]

    df['valid_swing_high'] = False
    for idx, valid in valid_swing_highs:
        df.at[idx, 'valid_swing_high'] = valid

    df['valid_swing_low'] = False
    for idx, valid in valid_swing_lows:
        df.at[idx, 'valid_swing_low'] = valid

    # Lọc loại bỏ các đỉnh liên tiếp không hợp lệ
    def filter_consecutive_highs():
        valid_highs = df.index[df['valid_swing_high']].tolist()
        valid_lows = df.index[df['valid_swing_low']].tolist()
        filtered_highs = []

        i = 0
        while i < len(valid_highs):
            curr_high = valid_highs[i]

            if i == len(valid_highs) - 1:
                filtered_highs.append(curr_high)
                break

            next_high = valid_highs[i + 1]
            lows_between = [low for low in valid_lows if curr_high < low < next_high]

            if lows_between:
                filtered_highs.append(curr_high)
                i += 1
            else:
                curr_high_price = df.at[curr_high, 'high']
                next_high_price = df.at[next_high, 'high']

                if curr_high_price > next_high_price:
                    filtered_highs.append(curr_high)
                    i += 2  # bỏ next_high
                else:
                    i += 1  # bỏ curr_high, giữ next_high

        if valid_highs[-1] not in filtered_highs:
            filtered_highs.append(valid_highs[-1])

        df['valid_swing_high'] = False
        df.loc[filtered_highs, 'valid_swing_high'] = True

    # Lọc loại bỏ các đáy liên tiếp không hợp lệ
    def filter_consecutive_lows():
        valid_lows = df.index[df['valid_swing_low']].tolist()
        valid_highs = df.index[df['valid_swing_high']].tolist()
        filtered_lows = []

        i = 0
        while i < len(valid_lows):
            curr_low = valid_lows[i]

            if i == len(valid_lows) - 1:
                filtered_lows.append(curr_low)
                break

            next_low = valid_lows[i + 1]
            highs_between = [high for high in valid_highs if curr_low < high < next_low]

            if highs_between:
                filtered_lows.append(curr_low)
                i += 1
            else:
                curr_low_price = df.at[curr_low, 'low']
                next_low_price = df.at[next_low, 'low']

                if curr_low_price < next_low_price:
                    filtered_lows.append(curr_low)
                    i += 2  # bỏ next_low
                else:
                    i += 1  # bỏ curr_low, giữ next_low

        if valid_lows[-1] not in filtered_lows:
            filtered_lows.append(valid_lows[-1])

        df['valid_swing_low'] = False
        df.loc[filtered_lows, 'valid_swing_low'] = True

    filter_consecutive_highs()
    filter_consecutive_lows()

    return df


def calculate_moving_averages(df, windows=[30, 60, 90]):
    """
    Tính các đường MA với list windows (mặc định 30, 60, 90).
    Trả về df với các cột MA tương ứng.
    """
    for w in windows:
        df[f'MA{w}'] = df['close'].rolling(window=w).mean()
    return df


In [29]:
# Tính swing high/low hợp lệ
eurusd_D = calculate_swing_high_low(eurusd_D, timeframe='D')
# Tính các đường MA
eurusd_D = calculate_moving_averages(eurusd_D, windows=[30, 60, 90])
eurusd_D['time'] = pd.to_datetime(eurusd_D['time'])


In [30]:
# Tính swing high/low hợp lệ
eurusd_W = calculate_swing_high_low(eurusd_W, timeframe='W')
# Tính các đường MA
eurusd_W = calculate_moving_averages(eurusd_W, windows=[30, 60, 90])
eurusd_W['time'] = pd.to_datetime(eurusd_W['time'])

In [31]:
# Tính swing high/low hợp lệ
eurusd_M = calculate_swing_high_low(eurusd_M, timeframe='M')
# Tính các đường MA
eurusd_M = calculate_moving_averages(eurusd_M, windows=[30, 60, 90])
eurusd_M['time'] = pd.to_datetime(eurusd_M['time'])

In [32]:
max_date = eurusd_D['time'].max()

In [33]:

def plot_candlestick(
    df,
    title="EURUSD Candlestick Chart",
    show_volume=False,
    show_swings=False,
    timeframe='D',  # D / W / M
    start_date=None,
    end_date=None
):
    """
    Vẽ biểu đồ nến cho EURUSD với các đường MA và tuỳ chọn volume + swing points.
    
    Args:
        df: DataFrame phải chứa các cột: time, open, high, low, close, MA30/60/90 (optional), volume (optional)
        title: Tiêu đề biểu đồ
        show_volume: Nếu True sẽ vẽ volume (dưới)
        show_swings: Nếu True sẽ vẽ các điểm swing high/low
        timeframe: 'D', 'W', 'M' (D: daily, weekly, monthly)
        start_date: chuỗi dạng 'YYYY-MM-DD' hoặc datetime để lọc ngày bắt đầu
        end_date: chuỗi dạng 'YYYY-MM-DD' hoặc datetime để lọc ngày kết thúc
    """

    # Lọc dữ liệu theo ngày nếu có
    if start_date:
        start_date = pd.to_datetime(start_date).tz_localize(df['time'].dt.tz)
    if end_date:
        end_date = pd.to_datetime(end_date).tz_localize(df['time'].dt.tz)

    # Lọc theo ngày nếu có
    if start_date or end_date:
        df = df.copy()
        if start_date:
            df = df[df['time'] >= start_date]
        if end_date:
            df = df[df['time'] <= end_date]

    fig = go.Figure()

    # Candlestick
    fig.add_trace(go.Candlestick(
        x=df['time'],
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        increasing_line_color='green',
        decreasing_line_color='black',
        name=f'{timeframe} Candlestick'
    ))

    # MA lines
    for ma, color in zip(['MA30', 'MA60', 'MA90'], ['blue', 'orange', 'green']):
        if ma in df.columns:
            fig.add_trace(go.Scatter(
                x=df['time'],
                y=df[ma],
                mode='lines',
                line=dict(color=color, width=1.5),
                name=ma
            ))

    # Swing High / Low
    if show_swings:
        if 'valid_swing_high' in df.columns:
            valid_sh = df[df['valid_swing_high']]
            fig.add_trace(go.Scatter(
                x=valid_sh['time'],
                y=valid_sh['high'],
                mode='markers',
                marker=dict(color='purple', size=10, symbol='star-triangle-up'),
                name='Swing High'
            ))
        if 'valid_swing_low' in df.columns:
            valid_sl = df[df['valid_swing_low']]
            fig.add_trace(go.Scatter(
                x=valid_sl['time'],
                y=valid_sl['low'],
                mode='markers',
                marker=dict(color='brown', size=10, symbol='star-triangle-down'),
                name='Swing Low'
            ))

    # Volume
    if show_volume and 'volume' in df.columns:
        volume_colors = ['green' if c >= o else 'black' for o, c in zip(df['open'], df['close'])]
        fig.add_trace(go.Bar(
            x=df['time'],
            y=df['volume'],
            name='Volume',
            marker_color=volume_colors,
            yaxis='y2',
            opacity=0.6
        ))

    # Layout
    fig.update_layout(
        title=title,
        xaxis=dict(
            domain=[0, 1],
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1Y", step="year", stepmode="backward"),
                    dict(count=2, label="2Y", step="year", stepmode="backward"),
                    dict(count=5, label="5Y", step="year", stepmode="backward"),
                    dict(count=10, label="10Y", step="year", stepmode="backward"),
                    dict(count=20, label="20Y", step="year", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"
        ),
        yaxis=dict(
            title='Price',
            domain=[0.3, 1] if show_volume else [0, 1],
            autorange=True
        ),
        yaxis2=dict(
            title='Volume',
            domain=[0, 0.25],
            autorange=True,
            showgrid=False
        ) if show_volume else None,
        xaxis_rangeslider_visible=False,
        template='plotly_white',
        height=500,
        width=1000,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
    )

    fig.show()



def plot_seasonal_tendency(df):
    df = df.copy()
    df['time'] = pd.to_datetime(df['time'])
    max_year = df['time'].dt.year.max()
    
    # Tính pct_change trên toàn bộ df, theo thứ tự thời gian
    df = df.sort_values('time').reset_index(drop=True)
    df['pct_change'] = df['close'].pct_change() * 100

    periods = [
        (max_year - 20, 'Last 20 Years'),
        (max_year - 10, 'Last 10 Years'),
        (max_year - 5,  'Last 5 Years'),
        (max_year - 2,  'Last 2 Years')
    ]

    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    data = []

    for start_year, label in periods:
        temp_df = df[df['time'].dt.year >= start_year].copy()
        temp_df['year'] = temp_df['time'].dt.year
        temp_df['month'] = temp_df['time'].dt.month

        pivot = temp_df.pivot(index='year', columns='month', values='pct_change')

        monthly_avg = pivot.mean(axis=0)

        for month_idx, avg_val in monthly_avg.items():
            data.append({
                'Month': month_names[month_idx - 1],
                'Average % Change': avg_val,
                'Period': label
            })

    df_plot = pd.DataFrame(data)

    fig = px.line(df_plot, x='Month', y='Average % Change', color='Period',
                  title='Seasonal Tendency by Month',
                  markers=True,
                  labels={'Average % Change': '% Change'})

    fig.update_layout(
        yaxis_tickformat='.2f',
        template='plotly_white',
        height=500,
        width=1000,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
    )

    fig.show()


In [12]:
import cot_reports as cot
legacy_fut, legacy_futopt, supplemental_futopt, disaggregated_fut, disaggregated_futopt, traders_fin_fut_fut, traders_fin_fut_futopt = cot.cot_all_reports()

eur_fx_futures_cme = legacy_fut[
    legacy_fut['Market and Exchange Names'].str.contains("EURO FX - CHICAGO MERCANTILE EXCHANGE", case=False)
]

legacy_fut
Selected: COT Legacy report. Futures only.
Stored the extracted file FUT86_16.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2017
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2018
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2019
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2020
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2021
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2022
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2023
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2024
Stored the file annual.txt in 

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

def plot_cot_index_with_extremes_plotly(df, start_date='2025-01-01', end_date=None, weeks=26, 
                                        upperExtreme=80, lowerExtreme=20):
    df = df.copy()
    df.rename(columns={
        'As of Date in Form YYYY-MM-DD': 'date',
        'Commercial Positions-Long (All)': 'commercial_long',
        'Commercial Positions-Short (All)': 'commercial_short',
        'Noncommercial Positions-Long (All)': 'noncommercial_long',
        'Noncommercial Positions-Short (All)': 'noncommercial_short',
        'Nonreportable Positions-Long (All)': 'retail_long',
        'Nonreportable Positions-Short (All)': 'retail_short',
        'Open Interest (All)': 'open_interest',
    }, inplace=True)

    df['date'] = pd.to_datetime(df['date'])

    # 1. Resample dữ liệu theo tuần (thứ Sáu)
    df_weekly = df.set_index('date').resample('W-FRI').last().dropna().reset_index()

    # 2. Tính net positions
    df_weekly['net_commercial'] = df_weekly['commercial_long'] - df_weekly['commercial_short']
    df_weekly['net_large'] = df_weekly['noncommercial_long'] - df_weekly['noncommercial_short']
    df_weekly['net_retail'] = df_weekly['retail_long'] - df_weekly['retail_short']

    # 3. Normalize theo open interest
    df_weekly['adj_commercial'] = df_weekly['net_commercial'] / df_weekly['open_interest'].replace(0, np.nan)
    df_weekly['adj_large'] = df_weekly['net_large'] / df_weekly['open_interest'].replace(0, np.nan)
    df_weekly['adj_retail'] = df_weekly['net_retail'] / df_weekly['open_interest'].replace(0, np.nan)

    # 4. Tính chỉ số COT Index
    def calc_index(adj_net_pos):
        min_val = adj_net_pos.rolling(window=weeks, min_periods=1).min()
        max_val = adj_net_pos.rolling(window=weeks, min_periods=1).max()
        idx = 100 * (adj_net_pos - min_val) / (max_val - min_val)
        idx[max_val == min_val] = np.nan
        return idx

    df_weekly['cot_index_commercial'] = calc_index(df_weekly['adj_commercial'])
    df_weekly['cot_index_large'] = calc_index(df_weekly['adj_large'])
    df_weekly['cot_index_retail'] = calc_index(df_weekly['adj_retail'])

    # 5. Lọc theo thời gian
    if end_date:
        mask = (df_weekly['date'] >= pd.to_datetime(start_date)) & (df_weekly['date'] <= pd.to_datetime(end_date))
    else:
        mask = df_weekly['date'] >= pd.to_datetime(start_date)
    df_ = df_weekly.loc[mask].copy()

    # 6. Vùng cực trị
    def find_regions(mask, dates):
        regions = []
        start = None
        for i, val in enumerate(mask):
            if val and start is None:
                start = dates.iat[i]
            elif not val and start is not None:
                end = dates.iat[i-1]
                regions.append((start, end))
                start = None
        if start is not None:
            regions.append((start, dates.iat[-1]))
        return regions

    bullish_mask = (df_['cot_index_commercial'] >= upperExtreme) & (df_['cot_index_retail'] <= lowerExtreme)
    bearish_mask = (df_['cot_index_commercial'] <= lowerExtreme) & (df_['cot_index_retail'] >= upperExtreme)

    bullish_regions = find_regions(bullish_mask.values, df_['date'])
    bearish_regions = find_regions(bearish_mask.values, df_['date'])

    # 7. Plot bằng Plotly
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df_['date'], y=df_['cot_index_commercial'],
                             mode='lines', name='Commercial', line=dict(color='green')))
    fig.add_trace(go.Scatter(x=df_['date'], y=df_['cot_index_large'],
                             mode='lines', name='Large Speculators', line=dict(color='red')))
    fig.add_trace(go.Scatter(x=df_['date'], y=df_['cot_index_retail'],
                             mode='lines', name='Retail', line=dict(color='blue')))

    # Vùng cực trị
    for start, end in bullish_regions:
        fig.add_vrect(x0=start, x1=end, fillcolor='blue', opacity=0.2, line_width=0)
    for start, end in bearish_regions:
        fig.add_vrect(x0=start, x1=end, fillcolor='red', opacity=0.2, line_width=0)

    # Đường ngưỡng
    fig.add_hline(y=upperExtreme, line_dash="dash", line_color="gray")
    fig.add_hline(y=lowerExtreme, line_dash="dash", line_color="gray")

    fig.update_layout(
        title=f"CoT Index with Extreme Zones ({start_date} to {end_date or 'latest'})",
        xaxis_title="Date",
        yaxis_title="CoT Index (%)",
        template="plotly_white",
        height=500,
        width=1000,
        legend=dict(x=0.01, y=0.99)
    )

    fig.show()


In [36]:
# import cot_reports as cot
# legacy_fut, legacy_futopt, supplemental_futopt, disaggregated_fut, disaggregated_futopt, traders_fin_fut_fut, traders_fin_fut_futopt = cot.cot_all_reports()

eur_fx_futures_cme = legacy_fut[
    legacy_fut['Market and Exchange Names'].str.contains("EURO FX - CHICAGO MERCANTILE EXCHANGE", case=False)
]

df = eur_fx_futures_cme[eur_fx_futures_cme['Market and Exchange Names'] == "EURO FX - CHICAGO MERCANTILE EXCHANGE"].copy()

In [38]:
plot_candlestick(eurusd_M, title="EURUSD - Monthly Chart", show_volume=True, show_swings=True, timeframe='M', start_date='2022-01-01', end_date='2025-07-21')
plot_candlestick(eurusd_W, title="EURUSD - Weekly Chart", show_volume=True, show_swings=True, timeframe='W', start_date='2022-01-01', end_date='2025-07-21')
plot_candlestick(eurusd_D, title="EURUSD - Daily Chart", show_volume=True, show_swings=True, timeframe='D', start_date='2023-01-01', end_date='2025-07-21')
plot_cot_index_with_extremes_plotly(df, weeks=26, upperExtreme=80, lowerExtreme=20, start_date='2023-01-01', end_date='2025-07-21', )
plot_seasonal_tendency(eurusd_M)


NameError: name 'px' is not defined

In [1]:
from cot_reports import cot_all_reports

cot_data_tuple = cot_all_reports(store_txt=True, verbose=True)
legacy_fut = cot_data_tuple[0]  # dùng cái này để truyền vào load_data()

legacy_fut
Selected: COT Legacy report. Futures only.
Stored the extracted file FUT86_16.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2017
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2018
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2019
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2020
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2021
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2022
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2023
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2024
Stored the file annual.txt in 

In [2]:
import pandas as pd
from price_loaders.tradingview import load_asset_price


def load_data(symbol: str, years: int, legacy_fut=None):
    if legacy_fut is None:
            raise ValueError("legacy_fut phải được truyền vào để tránh tải lại dữ liệu.")

    # Hợp lệ symbol
    valid_symbols = {"EURUSD", "GBPUSD", "USDJPY", "AUDUSD", "USDCHF", "NZDUSD", "USDCAD"}
    if symbol not in valid_symbols:
        raise ValueError(f"Symbol '{symbol}' không hợp lệ hoặc chưa được hỗ trợ.")
    
    # Thời gian tương ứng
    days = years * 252
    weeks = years * 52
    months = years * 12

    # Tải dữ liệu theo từng timeframe
    data_D = load_asset_price(symbol, days, "1D", None)
    data_W = load_asset_price(symbol, weeks, "1W", None)
    data_M = load_asset_price(symbol, months, "1M", None)
    
    # Kiểm tra dữ liệu không rỗng
    if data_D is None or data_D.empty:
        raise ValueError(f"Dữ liệu ngày cho '{symbol}' trống hoặc không hợp lệ.")
    if data_W is None or data_W.empty:
        raise ValueError(f"Dữ liệu tuần cho '{symbol}' trống hoặc không hợp lệ.")
    if data_M is None or data_M.empty:
        raise ValueError(f"Dữ liệu tháng cho '{symbol}' trống hoặc không hợp lệ.")
    
    # Chuyển đổi cột thời gian
    data_D['time'] = pd.to_datetime(data_D['time'])
    data_W['time'] = pd.to_datetime(data_W['time'])
    data_M['time'] = pd.to_datetime(data_M['time'])

    # CoT mapping
    cot_names = {
        'EURUSD': 'EURO FX - CHICAGO MERCANTILE EXCHANGE',
        'GBPUSD': 'BRITISH POUND - CHICAGO MERCANTILE EXCHANGE',
        'USDCHF': 'SWISS FRANC - CHICAGO MERCANTILE EXCHANGE',
        'USDJPY': 'JAPANESE YEN - CHICAGO MERCANTILE EXCHANGE',
        'NZDUSD': 'NZ DOLLAR - CHICAGO MERCANTILE EXCHANGE',
        'USDCAD': 'CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE',
        'AUDUSD': 'AUSTRALIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE'
    }


    if symbol in cot_names:
        cot_data = legacy_fut[
            legacy_fut['Market and Exchange Names'].str.contains(cot_names[symbol], case=False)
        ].copy()

        cot_data.rename(columns={
            'As of Date in Form YYYY-MM-DD': 'date',
            'Commercial Positions-Long (All)': 'commercial_long',
            'Commercial Positions-Short (All)': 'commercial_short',
            'Noncommercial Positions-Long (All)': 'noncommercial_long',
            'Noncommercial Positions-Short (All)': 'noncommercial_short',
            'Nonreportable Positions-Long (All)': 'retail_long',
            'Nonreportable Positions-Short (All)': 'retail_short',
            'Open Interest (All)': 'open_interest',
        }, inplace=True)

        cot_data['date'] = pd.to_datetime(cot_data['date'])

        # Lọc CoT theo số năm
        min_date = pd.Timestamp.today() - pd.DateOffset(years=years)
        cot_data = cot_data[cot_data['date'] >= min_date].copy()
    else:
        cot_data = None

    return {
        "1D_data": data_D,
        "1W_data": data_W,
        "1M_data": data_M,
        "COT_data": cot_data
    }


In [39]:
df = load_data(symbol="EURUSD", years=5, legacy_fut = legacy_fut)

In [40]:
df

{'1D_data':                           time     open     high      low    close    volume
 0    2020-09-16 04:00:00+07:00  1.18461  1.18827  1.17878  1.18155  393681.0
 1    2020-09-17 04:00:00+07:00  1.18155  1.18524  1.17372  1.18480  369279.0
 2    2020-09-18 04:00:00+07:00  1.18480  1.18705  1.18261  1.18374  274561.0
 3    2020-09-21 04:00:00+07:00  1.18405  1.18719  1.17316  1.17706  383649.0
 4    2020-09-22 04:00:00+07:00  1.17706  1.17737  1.16916  1.17077  382516.0
 ...                        ...      ...      ...      ...      ...       ...
 1255 2025-07-15 04:00:00+07:00  1.16641  1.16926  1.15928  1.16009  213759.0
 1256 2025-07-16 04:00:00+07:00  1.16009  1.17213  1.15623  1.16399  356498.0
 1257 2025-07-17 04:00:00+07:00  1.16399  1.16424  1.15564  1.15950  200336.0
 1258 2025-07-18 04:00:00+07:00  1.15950  1.16718  1.15906  1.16251  192096.0
 1259 2025-07-21 04:00:00+07:00  1.16275  1.16522  1.16145  1.16420   72719.0
 
 [1260 rows x 6 columns],
 '1W_data':              

In [97]:
daily_df = df["1D_data"]
weekly_df = df["1W_data"]
monthly_df = df["1M_data"]
cot_df = df["COT_data"]


,time,open,high,low,close,volume
0,2020-09-16 04:00:00+07:00,1.18461,1.18827,1.17878,1.18155,393681.0
1,2020-09-17 04:00:00+07:00,1.18155,1.18524,1.17372,1.18480,369279.0
2,2020-09-18 04:00:00+07:00,1.18480,1.18705,1.18261,1.18374,274561.0
3,2020-09-21 04:00:00+07:00,1.18405,1.18719,1.17316,1.17706,383649.0
4,2020-09-22 04:00:00+07:00,1.17706,1.17737,1.16916,1.17077,382516.0
...,...,...,...,...,...,...
1255,2025-07-15 04:00:00+07:00,1.16641,1.16926,1.15928,1.16009,213759.0
1256,2025-07-16 04:00:00+07:00,1.16009,1.17213,1.15623,1.16399,356498.0
1257,2025-07-17 04:00:00+07:00,1.16399,1.16424,1.15564,1.15950,200336.0
1258,2025-07-18 04:00:00+07:00,1.15950,1.16718,1.15906,1.16251,192096.0


In [54]:
# Tính swing high/low hợp lệ
daily_df = calculate_swing_high_low(daily_df, timeframe='D')
# Tính các đường MA
daily_df = calculate_moving_averages(daily_df, windows=[30, 60, 90])
daily_df['time'] = pd.to_datetime(daily_df['time'])

# Lọc dữ liệu từ 2023-01-01 đến 2024-12-31
start_date = '2023-01-01'
end_date = '2024-12-31'

daily_df = daily_df[(daily_df['time'] >= start_date) & (daily_df['time'] <= end_date)]

In [55]:
daily_df

,time,open,high,low,close,volume,swing_high,swing_low,valid_swing_high,valid_swing_low,MA30,MA60,MA90
0,2023-01-02 05:00:00+07:00,1.06994,1.06994,1.06495,1.06647,50193.0,False,False,False,False,NaN,NaN,NaN
1,2023-01-03 05:00:00+07:00,1.06647,1.06831,1.05194,1.05467,385015.0,False,False,False,False,NaN,NaN,NaN
2,2023-01-04 05:00:00+07:00,1.05467,1.06351,1.05403,1.06019,385003.0,False,False,False,False,NaN,NaN,NaN
3,2023-01-05 05:00:00+07:00,1.06019,1.06314,1.05149,1.05214,347098.0,False,False,False,False,NaN,NaN,NaN
4,2023-01-06 05:00:00+07:00,1.05214,1.06480,1.04819,1.06392,349636.0,False,True,False,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,2024-12-23 05:00:00+07:00,1.04266,1.04450,1.03839,1.04045,185810.0,False,False,False,False,1.050986,1.069075,1.083100
516,2024-12-24 05:00:00+07:00,1.04045,1.04096,1.03831,1.03912,124562.0,False,False,False,False,1.050214,1.067948,1.082280
517,2024-12-26 05:00:00+07:00,1.03912,1.04298,1.03901,1.04203,115136.0,False,False,False,False,1.049741,1.066908,1.081470
518,2024-12-27 05:00:00+07:00,1.04203,1.04438,1.04046,1.04205,163688.0,False,False,False,False,1.049378,1.065892,1.080701


In [ ]:
def determine_ma_trend(ma30, ma60, ma90):
    if ma30 > ma90 and ma60 > ma90:
        return 'Uptrend'
    elif ma30 < ma90 and ma60 < ma90:
        return 'Downtrend'
    elif (ma30 > ma90 and ma60 < ma90) or (ma30 < ma90 and ma60 > ma90):
        return 'Sideways'
    else:
        return 'Unknown'

def print_last_month_ma_trend(df):
    # Đảm bảo cột 'time' là datetime
    df['time'] = pd.to_datetime(df['time'])

    # Lấy tháng hiện tại và năm hiện tại
    today = df['time'].iloc[-1]
    current_month = today.month
    current_year = today.year

    # Tính tháng và năm trước
    if current_month == 1:
        last_month = 12
        last_month_year = current_year - 1
    else:
        last_month = current_month - 1
        last_month_year = current_year

    # Lọc các dòng thuộc tháng trước
    last_month_df = df[
        (df['time'].dt.month == last_month) & 
        (df['time'].dt.year == last_month_year)
    ].dropna(subset=['MA30', 'MA60', 'MA90'])

    if last_month_df.empty:
        print("❌ Không có dữ liệu cho tháng trước.")
        return

    # Lấy ngày cuối cùng trong tháng trước
    last_row = last_month_df.iloc[-1]
    date = last_row['time'].strftime('%Y-%m-%d')
    close = f"{last_row['close']:.5f}"
    ma30 = last_row['MA30']
    ma60 = last_row['MA60']
    ma90 = last_row['MA90']
    trend = determine_ma_trend(ma30, ma60, ma90)

    # In báo cáo
    print("====== BÁO CÁO XU HƯỚNG MA - Cuối Tháng Trước ======")
    print(f"{'Date':<12} | {'Close':>7} | {'MA30':>7} | {'MA60':>7} | {'MA90':>7} | {'Trend':<10}")
    print("-" * 65)
    print(f"{date:<12} | {close:>7} | {ma30:7.5f} | {ma60:7.5f} | {ma90:7.5f} | {trend:<10}")



In [57]:
print_last_month_ma_trend(daily_df)

2024-12-30 05:00:00+07:00
11
====== BÁO CÁO XU HƯỚNG MA - Cuối Tháng Trước ======
Date         |   Close |    MA30 |    MA60 |    MA90 | Trend     
-----------------------------------------------------------------
2024-11-29   | 1.05741 | 1.06822 | 1.08581 | 1.09096 | Downtrend 


In [112]:
def determine_structure_trend_verbose(df):
    df['time'] = pd.to_datetime(df['time'])

    # Loại bỏ tháng hiện tại
    now = df['time'].iloc[-1]
    df_filtered = df[(df['time'].dt.month != now.month) | (df['time'].dt.year != now.year)]

    # Lấy các swing high/low hợp lệ
    valid_highs = df_filtered[df_filtered['valid_swing_high']].reset_index()
    valid_lows = df_filtered[df_filtered['valid_swing_low']].reset_index()

    # Biến lưu kết quả uptrend / downtrend (nếu có)
    uptrend_info = None
    downtrend_info = None

    # 1️⃣ Kiểm tra Uptrend
    if len(valid_highs) >= 2:
        last_high = valid_highs.iloc[-1]
        prev_high = valid_highs.iloc[-2]
        range_start = prev_high['index']
        range_end = last_high['index']

        middle_lows = df_filtered[
            (df_filtered['valid_swing_low']) &
            (df_filtered.index > range_start) & (df_filtered.index < range_end)
        ]
        if not middle_lows.empty:
            middle_low = middle_lows.iloc[-1]
            uptrend_info = {
                'type': 'Uptrend',
                'time': last_high['time'],
                'text': (
                    "📈 Xu hướng: **Uptrend**\n"
                    f"   🔹 Đỉnh trước: {prev_high['time'].strftime('%Y-%m-%d')} | Giá: {prev_high['high']:.5f}\n"
                    f"   🔹 Đỉnh sau  : {last_high['time'].strftime('%Y-%m-%d')} | Giá: {last_high['high']:.5f}\n"
                    f"   🔸 Đáy giữa  : {middle_low['time'].strftime('%Y-%m-%d')} | Giá: {middle_low['low']:.5f}"
                )
            }

    # 2️⃣ Kiểm tra Downtrend
    if len(valid_lows) >= 2:
        last_low = valid_lows.iloc[-1]
        prev_low = valid_lows.iloc[-2]
        range_start = prev_low['index']
        range_end = last_low['index']

        middle_highs = df_filtered[
            (df_filtered['valid_swing_high']) &
            (df_filtered.index > range_start) & (df_filtered.index < range_end)
        ]
        if not middle_highs.empty:
            middle_high = middle_highs.iloc[-1]
            downtrend_info = {
                'type': 'Downtrend',
                'time': last_low['time'],
                'text': (
                    "📉 Xu hướng: **Downtrend**\n"
                    f"   🔸 Đáy trước: {prev_low['time'].strftime('%Y-%m-%d')} | Giá: {prev_low['low']:.5f}\n"
                    f"   🔸 Đáy sau  : {last_low['time'].strftime('%Y-%m-%d')} | Giá: {last_low['low']:.5f}\n"
                    f"   🔹 Đỉnh giữa: {middle_high['time'].strftime('%Y-%m-%d')} | Giá: {middle_high['high']:.5f}"
                )
            }

    # ✅ So sánh thời gian để xác định xu hướng cuối cùng
    if uptrend_info and downtrend_info:
        if uptrend_info['time'] > downtrend_info['time']:
            print(uptrend_info['text'])
            return uptrend_info['type']
        else:
            print(downtrend_info['text'])
            return downtrend_info['type']
    elif uptrend_info:
        print(uptrend_info['text'])
        return uptrend_info['type']
    elif downtrend_info:
        print(downtrend_info['text'])
        return downtrend_info['type']
    else:
        print("➖ Xu hướng: Sideways / Không xác định")
        return 'Sideways or Indeterminate'

In [114]:
trend = determine_structure_trend_verbose(daily_df)
print("📈 Xu hướng theo cấu trúc swing:", trend)

KeyError: 'valid_swing_high'

In [96]:
cot_df = df["COT_data"]
cot_df

,Market and Exchange Names,As of Date in Form YYMMDD,date,CFTC Contract Market Code,CFTC Market Code in Initials,CFTC Region Code,CFTC Commodity Code,open_interest,noncommercial_long,noncommercial_short,...,Concentration-Gross LT =8 TDR-Long (Other),Concentration-Gross LT =8 TDR-Short(Other),Concentration-Net LT =4 TDR-Long (Other),Concentration-Net LT =4 TDR-Short (Other),Concentration-Net LT =8 TDR-Long (Other),Concentration-Net LT =8 TDR-Short (Other),Contract Units,CFTC Contract Market Code (Quotes),CFTC Market Code in Initials (Quotes),CFTC Commodity Code (Quotes)
11133,EURO FX - CHICAGO MERCANTILE EXCHANGE,201229,2020-12-29,099741,CME,0,99,671643,222848,79772,...,0.0,0.0,0.0,0.0,0.0,0.0,"(CONTRACTS OF EUR 125,000)",099741,CME,99
11134,EURO FX - CHICAGO MERCANTILE EXCHANGE,201221,2020-12-21,099741,CME,0,99,659499,222443,78541,...,0.0,0.0,0.0,0.0,0.0,0.0,"(CONTRACTS OF EUR 125,000)",099741,CME,99
11135,EURO FX - CHICAGO MERCANTILE EXCHANGE,201215,2020-12-15,099741,CME,0,99,657046,218710,76877,...,0.0,0.0,0.0,0.0,0.0,0.0,"(CONTRACTS OF EUR 125,000)",099741,CME,99
11136,EURO FX - CHICAGO MERCANTILE EXCHANGE,201208,2020-12-08,099741,CME,0,99,683351,222521,66092,...,0.0,0.0,0.0,0.0,0.0,0.0,"(CONTRACTS OF EUR 125,000)",099741,CME,99
11137,EURO FX - CHICAGO MERCANTILE EXCHANGE,201201,2020-12-01,099741,CME,0,99,654257,207302,67408,...,0.0,0.0,0.0,0.0,0.0,0.0,"(CONTRACTS OF EUR 125,000)",099741,CME,99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7224,EURO FX - CHICAGO MERCANTILE EXCHANGE,250204,2025-02-04,099741,CME,0,99,616623,162554,221168,...,0.0,0.0,0.0,0.0,0.0,0.0,"(CONTRACTS OF EUR 125,000)",099741,CME,99
7225,EURO FX - CHICAGO MERCANTILE EXCHANGE,250128,2025-01-28,099741,CME,0,99,618908,153660,220264,...,0.0,0.0,0.0,0.0,0.0,0.0,"(CONTRACTS OF EUR 125,000)",099741,CME,99
7226,EURO FX - CHICAGO MERCANTILE EXCHANGE,250121,2025-01-21,099741,CME,0,99,609492,167665,230151,...,0.0,0.0,0.0,0.0,0.0,0.0,"(CONTRACTS OF EUR 125,000)",099741,CME,99
7227,EURO FX - CHICAGO MERCANTILE EXCHANGE,250114,2025-01-14,099741,CME,0,99,604225,162760,223157,...,0.0,0.0,0.0,0.0,0.0,0.0,"(CONTRACTS OF EUR 125,000)",099741,CME,99


In [98]:
cot_df['date'] = pd.to_datetime(cot_df['date'])

# Lọc từ 2023 đến 2024
cot_df = cot_df[(cot_df['date'] >= '2024-01-01') & (cot_df['date'] <= '2024-12-31')]

In [99]:
cot_df

,Market and Exchange Names,As of Date in Form YYMMDD,date,CFTC Contract Market Code,CFTC Market Code in Initials,CFTC Region Code,CFTC Commodity Code,open_interest,noncommercial_long,noncommercial_short,...,Concentration-Gross LT =8 TDR-Long (Other),Concentration-Gross LT =8 TDR-Short(Other),Concentration-Net LT =4 TDR-Long (Other),Concentration-Net LT =4 TDR-Short (Other),Concentration-Net LT =8 TDR-Long (Other),Concentration-Net LT =8 TDR-Short (Other),Contract Units,CFTC Contract Market Code (Quotes),CFTC Market Code in Initials (Quotes),CFTC Commodity Code (Quotes)
13693,EURO FX - CHICAGO MERCANTILE EXCHANGE,241231,2024-12-31,099741,CME,0,99,609159,168806,238370,...,0.0,0.0,0.0,0.0,0.0,0.0,"(CONTRACTS OF EUR 125,000)",099741,CME,99
13694,EURO FX - CHICAGO MERCANTILE EXCHANGE,241224,2024-12-24,099741,CME,0,99,597583,159471,227978,...,0.0,0.0,0.0,0.0,0.0,0.0,"(CONTRACTS OF EUR 125,000)",099741,CME,99
13695,EURO FX - CHICAGO MERCANTILE EXCHANGE,241217,2024-12-17,099741,CME,0,99,584000,152671,218566,...,0.0,0.0,0.0,0.0,0.0,0.0,"(CONTRACTS OF EUR 125,000)",099741,CME,99
13696,EURO FX - CHICAGO MERCANTILE EXCHANGE,241210,2024-12-10,099741,CME,0,99,727276,157375,232948,...,0.0,0.0,0.0,0.0,0.0,0.0,"(CONTRACTS OF EUR 125,000)",099741,CME,99
13697,EURO FX - CHICAGO MERCANTILE EXCHANGE,241203,2024-12-03,099741,CME,0,99,675059,167693,225182,...,0.0,0.0,0.0,0.0,0.0,0.0,"(CONTRACTS OF EUR 125,000)",099741,CME,99
13698,EURO FX - CHICAGO MERCANTILE EXCHANGE,241126,2024-11-26,099741,CME,0,99,665553,156334,212343,...,0.0,0.0,0.0,0.0,0.0,0.0,"(CONTRACTS OF EUR 125,000)",099741,CME,99
13699,EURO FX - CHICAGO MERCANTILE EXCHANGE,241119,2024-11-19,099741,CME,0,99,658774,154305,196862,...,0.0,0.0,0.0,0.0,0.0,0.0,"(CONTRACTS OF EUR 125,000)",099741,CME,99
13700,EURO FX - CHICAGO MERCANTILE EXCHANGE,241112,2024-11-12,099741,CME,0,99,637105,160003,167440,...,0.0,0.0,0.0,0.0,0.0,0.0,"(CONTRACTS OF EUR 125,000)",099741,CME,99
13701,EURO FX - CHICAGO MERCANTILE EXCHANGE,241105,2024-11-05,099741,CME,0,99,645597,159900,181553,...,0.0,0.0,0.0,0.0,0.0,0.0,"(CONTRACTS OF EUR 125,000)",099741,CME,99
13702,EURO FX - CHICAGO MERCANTILE EXCHANGE,241029,2024-10-29,099741,CME,0,99,661675,159313,209617,...,0.0,0.0,0.0,0.0,0.0,0.0,"(CONTRACTS OF EUR 125,000)",099741,CME,99


In [124]:
import pandas as pd
import numpy as np

def determine_cot_trend_by_month_loop(cot_df, upperExtreme=80, lowerExtreme=20, weeks=26):
    # Chuẩn hóa cột, convert date
    cot_df = cot_df.rename(columns={
        'As of Date in Form YYYY-MM-DD': 'date',
        'Commercial Positions-Long (All)': 'commercial_long',
        'Commercial Positions-Short (All)': 'commercial_short',
        'Noncommercial Positions-Long (All)': 'noncommercial_long',
        'Noncommercial Positions-Short (All)': 'noncommercial_short',
        'Nonreportable Positions-Long (All)': 'retail_long',
        'Nonreportable Positions-Short (All)': 'retail_short',
        'Open Interest (All)': 'open_interest',
    })
    cot_df['date'] = pd.to_datetime(cot_df['date'])
    
    # Resample tuần, lấy giá cuối tuần (Friday)
    df_weekly = cot_df.set_index('date').resample('W-FRI').last().dropna().reset_index()

    # Tính net positions
    df_weekly['net_commercial'] = df_weekly['commercial_long'] - df_weekly['commercial_short']
    df_weekly['net_large'] = df_weekly['noncommercial_long'] - df_weekly['noncommercial_short']
    df_weekly['net_retail'] = df_weekly['retail_long'] - df_weekly['retail_short']
    df_weekly['open_interest'] = df_weekly['open_interest'].replace(0, np.nan)
    
    # Chuẩn hóa net positions
    df_weekly['adj_commercial'] = df_weekly['net_commercial'] / df_weekly['open_interest']
    df_weekly['adj_large'] = df_weekly['net_large'] / df_weekly['open_interest']
    df_weekly['adj_retail'] = df_weekly['net_retail'] / df_weekly['open_interest']

    # Hàm tính COT index theo rolling window
    def calc_index(adj_net_pos):
        min_val = adj_net_pos.rolling(window=weeks, min_periods=1).min()
        max_val = adj_net_pos.rolling(window=weeks, min_periods=1).max()
        idx = 100 * (adj_net_pos - min_val) / (max_val - min_val)
        idx[max_val == min_val] = np.nan
        return idx

    df_weekly['cot_index_commercial'] = calc_index(df_weekly['adj_commercial'])
    df_weekly['cot_index_large'] = calc_index(df_weekly['adj_large'])
    df_weekly['cot_index_retail'] = calc_index(df_weekly['adj_retail'])

    # Xác định tháng cuối cùng trong dữ liệu
    last_date = df_weekly['date'].max()
    current_year = last_date.year
    current_month = last_date.month
    
    # Hàm để kiểm tra xu hướng trong 1 tháng (lấy giá cuối cùng của tháng đó)
    def check_trend_for_month(year, month):
        month_data = df_weekly[(df_weekly['date'].dt.year == year) & (df_weekly['date'].dt.month == month)]
        if month_data.empty:
            return None
        last_row = month_data.iloc[-1]
        cot_com = last_row['cot_index_commercial']
        cot_retail = last_row['cot_index_retail']

        if cot_com >= upperExtreme and cot_retail <= lowerExtreme:
            return 'Bullish (Uptrend)'
        elif cot_com <= lowerExtreme and cot_retail >= upperExtreme:
            return 'Bearish (Downtrend)'
        else:
            return None

    # Vòng lặp lùi tháng
    for i in range(1, 100):  # Giới hạn lùi 100 tháng tránh vô hạn
        # Tính tháng và năm lùi i tháng
        month_to_check = current_month - i
        year_to_check = current_year
        while month_to_check <= 0:
            month_to_check += 12
            year_to_check -= 1

        trend = check_trend_for_month(year_to_check, month_to_check)
        if trend is not None:
            print(f"Xu hướng đầu tiên tìm được tại tháng {year_to_check}-{month_to_check:02d}: {trend}")
            return trend
    
    print("Không tìm thấy xu hướng rõ ràng trong dữ liệu.")
    return None


In [125]:
determine_cot_trend_by_month_loop(cot_df)

Xu hướng đầu tiên tìm được tại tháng 2024-12: Bullish (Uptrend)


'Bullish (Uptrend)'

In [126]:
print("Max date in cot_df:", cot_df['date'].max())

Max date in cot_df: 2024-12-31 00:00:00
